# ⏰ Orchestration de Pipelines Data

Ce module présente les outils d'**orchestration** pour automatiser l'exécution de vos pipelines de données.

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi le module `11_pyspark_for_data_engineering` |
| ✅ Requis | Comprendre les pipelines ETL |
| ✅ Requis | Maîtriser Python (modules 04-05) |
| ✅ Requis | Connaître les bases de Linux (ligne de commande) |

## 🎯 Objectifs du module

À la fin de ce notebook, tu seras capable de :

- ✅ Comprendre ce qu'est l'orchestration de pipelines
- ✅ Utiliser le Planificateur Windows (niveau débutant)
- ✅ Configurer des tâches avec Crontab (niveau intermédiaire)
- ✅ Créer des DAGs avec Apache Airflow (niveau avancé)
- ✅ Choisir le bon outil selon ton besoin

---

## 🎯 L'orchestration dans l'écosystème Data Engineering

Tu as appris à créer des pipelines ETL avec PySpark. Mais comment les **automatiser** pour qu'ils s'exécutent régulièrement sans intervention manuelle ?

### Le problème

```
Sans orchestration :
┌─────────────────────────────────────────────────────────────┐
│                                                             │
│   😰 "Il faut que je lance mon script tous les jours..."   │
│   😰 "J'ai oublié de lancer le pipeline hier !"            │
│   😰 "Le script B a planté car A n'avait pas fini..."      │
│                                                             │
└─────────────────────────────────────────────────────────────┘

Avec orchestration :
┌─────────────────────────────────────────────────────────────┐
│                                                             │
│   ✅ Scripts exécutés automatiquement                       │
│   ✅ Alertes en cas d'échec                                 │
│   ✅ Dépendances respectées (A → B → C)                     │
│   ✅ Logs et monitoring centralisés                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

### Position dans le pipeline Data

```
┌─────────────────────────────────────────────────────────────────┐
│                     PIPELINE DATA                               │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   Sources        ETL              Destination                   │
│   ────────       ───              ───────────                   │
│                                                                 │
│   APIs     ─┐                 ┌─►  Data Warehouse               │
│   Fichiers ─┼──►  PySpark  ───┼─►  Data Lake                    │
│   BDD      ─┘                 └─►  Dashboard                    │
│                                                                 │
│            ▲                                                    │
│            │                                                    │
│   ┌────────┴────────┐                                           │
│   │  ORCHESTRATION  │  ◄── Quand ? Dans quel ordre ?           │
│   │  (Airflow/Cron) │                                           │
│   └─────────────────┘                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## 📊 Comparaison rapide des outils

| Critère | Windows Task | Crontab | Airflow |
|---------|--------------|---------|----------|
| **Facilité** | ⭐⭐⭐ | ⭐⭐ | ⭐ |
| **Interface** | GUI | CLI | Web |
| **Dépendances** | ❌ | ❌ | ✅ |
| **Monitoring** | Basique | Non | Complet |
| **Retry auto** | ❌ | ❌ | ✅ |
| **Idéal pour** | 1-5 scripts | 5-15 scripts | 10+ pipelines |

---

# 🪟 Niveau 1 : Planificateur Windows

## C'est quoi ?

Un outil **intégré à Windows** pour exécuter des programmes automatiquement.

## Comment l'utiliser ?

### Ouvrir le planificateur :
```
Windows + R → Taper 'taskschd.msc' → Entrée
```

### Créer une tâche :
1. **Action** → Créer une tâche de base
2. **Nom** : "Mon script quotidien"
3. **Déclencheur** : Quotidien à 2h du matin
4. **Action** : Démarrer `python.exe` avec `C:\scripts\mon_script.py`
5. **Terminer**

✅ **Voilà !** Votre script s'exécutera automatiquement tous les jours à 2h.

## ✅ Forces

✅ **Très facile** - Interface graphique intuitive  
✅ **Déjà installé** - Pas de setup  
✅ **Parfait pour débuter** - Pas de code complexe  

## ❌ Faiblesses

❌ **Pas de dépendances** - Si tâche A doit finir avant tâche B → compliqué  
❌ **Monitoring limité** - Difficile de voir l'état global  
❌ **Windows uniquement** - Ne fonctionne pas sur Linux  

## Quand l'utiliser ?

✅ **OUI** : Vous avez 1-5 scripts simples sur Windows  
❌ **NON** : Vous avez besoin que script B attende script A

---

# 🐧 Niveau 2 : Crontab (Linux/Mac)

## C'est quoi ?

Le **planificateur standard** sur Linux/Mac.

## Syntaxe de base

```bash
minute heure jour mois jour_semaine commande
```

### Exemples simples :

```bash
# Tous les jours à 2h du matin
0 2 * * * python3 /home/user/script.py

# Toutes les heures
0 * * * * python3 /home/user/hourly.py

# Lundi à vendredi à 9h
0 9 * * 1-5 python3 /home/user/weekday.py
```

## Comment l'utiliser ?

```bash
# Éditer votre crontab
crontab -e

# Ajouter vos lignes
0 2 * * * python3 /home/user/backup.py

# Sauvegarder et quitter
# ✅ C'est fait !
```

## ✅ Forces

✅ **Universel** - Sur TOUS les serveurs Linux  
✅ **Très léger** - Presque pas de ressources  
✅ **Simple** - Une ligne = une tâche  
✅ **Gratuit** - Déjà installé  

## ❌ Faiblesses

❌ **Pas de dépendances** - Même problème que Windows  
❌ **Pas de monitoring** - Aucune interface  
❌ **Pas de retry** - Si ça échoue, il faut attendre le prochain run  
❌ **Logs manuels** - Il faut les gérer soi-même  

## Quand l'utiliser ?

✅ **OUI** : Serveur Linux, 5-15 scripts indépendants  
❌ **NON** : Scripts avec dépendances complexes

---

# 🌬️ Niveau 3 : Apache Airflow

## C'est quoi ?

Un **orchestrateur professionnel** pour gérer des workflows complexes.

### La grande différence :

```python
# Avec Cron (problème) :
0 2 * * * python extract.py
30 2 * * * python transform.py  # Et si extract prend plus de 30min ?

# Avec Airflow (solution) :
extract >> transform >> load  # transform ATTEND extract !
```

## Installation

```bash
pip install apache-airflow
airflow db init
airflow webserver --port 8080  # Terminal 1
airflow scheduler              # Terminal 2
```

Accéder : **http://localhost:8080**

## Exemple simple

In [ ]:
# Fichier : ~/airflow/dags/mon_pipeline.py

from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime

# Définir le workflow
dag = DAG(
    'mon_pipeline',
    start_date=datetime(2024, 1, 1),
    schedule_interval='@daily',  # Tous les jours
    catchup=False
)

# Fonction Python
def extract():
    print("📥 Extraction des données")
    return "data.csv"

def transform():
    print("🔄 Transformation")
    return "data_clean.csv"

def load():
    print("💾 Chargement en base")

# Créer les tâches
extract_task = PythonOperator(task_id='extract', python_callable=extract, dag=dag)
transform_task = PythonOperator(task_id='transform', python_callable=transform, dag=dag)
load_task = PythonOperator(task_id='load', python_callable=load, dag=dag)

# Définir les dépendances
extract_task >> transform_task >> load_task

# ✅ Airflow garantit que transform attend extract !

## ✅ Forces

✅ **Gestion des dépendances** - `A >> B` = B attend A  
✅ **Retry automatique** - Réessaie en cas d'échec  
✅ **Interface web** - Visualisation complète  
✅ **Monitoring** - Logs centralisés  
✅ **Alertes** - Email/Slack en cas d'échec  
✅ **Scalable** - Gère 100+ pipelines  

## ❌ Faiblesses

❌ **Complexe** - Courbe d'apprentissage  
❌ **Ressources** - Besoin de 4-8 GB RAM  
❌ **Overkill** - Pour 1-3 scripts simples  
❌ **Setup** - Installation et configuration  

## Quand l'utiliser ?

✅ **OUI** : 10+ pipelines, dépendances, production  
❌ **NON** : 1-5 scripts simples sans dépendances

---

# 📊 Tableau de décision

## Quel outil choisir ?

| Situation | Outil recommandé |
|-----------|------------------|
| J'ai 1-3 scripts sur Windows | 🪟 **Task Scheduler** |
| J'ai 1-3 scripts sur Linux | 🐧 **Crontab** |
| J'ai 5-10 scripts indépendants | 🐧 **Crontab** |
| J'ai 10+ scripts avec dépendances | 🌬️ **Airflow** |
| Script B doit attendre script A | 🌬️ **Airflow** |
| Je veux un dashboard | 🌬️ **Airflow** |
| Je débute en automatisation | 🪟 **Task Scheduler** |
| Production critique | 🌬️ **Airflow** |

## Progression naturelle

```
1. Débutez avec Task Scheduler ou Cron
2. Quand vous avez 5+ scripts → Pensez à migrer
3. Quand vous avez des dépendances → Migrez vers Airflow
```

---

# 🎓 Résumé

## Points clés

### 🪟 Windows Task Scheduler
- **Pour qui** : Débutants sur Windows
- **Force** : Très facile (GUI)
- **Faiblesse** : Pas de dépendances
- **Limite** : 5 scripts max

### 🐧 Crontab
- **Pour qui** : Utilisateurs Linux
- **Force** : Universel, léger
- **Faiblesse** : Pas de monitoring
- **Limite** : 15 scripts max

### 🌬️ Airflow
- **Pour qui** : Production
- **Force** : Dépendances, monitoring
- **Faiblesse** : Complexe, ressources
- **Limite** : Aucune (scalable)

## Ressources

- **Crontab** : [crontab.guru](https://crontab.guru/) (tester expressions)
- **Airflow** : [airflow.apache.org](https://airflow.apache.org/)

---

# 🧪 Quiz Final

### ❓ Q1. Vous avez 2 scripts Python à exécuter tous les jours sur Windows. Quel outil ?
a) Airflow  
b) Crontab  
c) Windows Task Scheduler  
d) Kubernetes

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : c** – Pour 2 scripts simples sur Windows, utilisez le **Planificateur de tâches** : facile, natif, gratuit.
</details>

---

### ❓ Q2. Expression crontab pour "tous les lundis à 9h" ?
a) `0 9 * * 1`  
b) `9 0 * * 1`  
c) `* 9 * * 1`  
d) `0 9 1 * *`

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : a** – Format : `minute heure jour mois jour_semaine`. `0 9 * * 1` = 0 minutes, 9h, lundi (1).
</details>

---

### ❓ Q3. Principale faiblesse de Cron et Task Scheduler ?
a) Trop lents  
b) Ne gèrent pas les dépendances entre tâches  
c) Payants  
d) Difficiles à utiliser

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : b** – Le gros problème : **pas de dépendances**. Si A doit finir avant B, il faut gérer ça manuellement.
</details>

---

### ❓ Q4. Dans Airflow, que signifie `task_a >> task_b` ?
a) task_a et task_b en parallèle  
b) task_b attend que task_a finisse  
c) task_a attend task_b  
d) Erreur de syntaxe

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : b** – `A >> B` signifie "B **attend** que A soit terminé". C'est la gestion des dépendances !
</details>

---

### ❓ Q5. Principal avantage de Crontab ?
a) Interface graphique  
b) Extrêmement léger et déjà installé sur Linux  
c) Gère les dépendances  
d) Monitoring complet

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : b** – Cron est **ultra-léger** (quelques KB de RAM), **préinstallé** sur tous les Linux, et **gratuit**.
</details>

---

### ❓ Q6. Que se passe-t-il si un script cron échoue ?
a) Cron réessaie automatiquement  
b) Cron envoie une alerte  
c) Rien, il faut attendre le prochain run  
d) Tous les scripts s'arrêtent

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : c** – Cron **ne réessaie PAS** automatiquement. Si ça échoue, vous attendez le prochain run.
</details>

---

### ❓ Q7. RAM minimum recommandée pour Airflow en production ?
a) 512 MB  
b) 2 GB  
c) 8 GB  
d) 32 GB

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : c** – Pour Airflow en production, comptez **8 GB RAM minimum** (et 4 CPU cores).
</details>

---

### ❓ Q8. Quand NE PAS utiliser Airflow ?
a) 10+ pipelines complexes  
b) 1-3 scripts simples  
c) Production critique  
d) Besoin de monitoring

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : b** – Pour **1-3 scripts simples**, Airflow est **overkill**. Utilisez Cron ou Task Scheduler à la place.
</details>

---

### ❓ Q9. Principal avantage d'Airflow sur Cron ?
a) Plus léger  
b) Plus facile  
c) Gestion native des dépendances  
d) Gratuit

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : c** – L'**avantage clé** d'Airflow : **gestion des dépendances**. `A >> B` garantit que B attend A.
</details>

---

### ❓ Q10. Vous avez 15 scripts avec des dépendances. Quel outil ?
a) Windows Task Scheduler  
b) Crontab  
c) Apache Airflow  
d) Aucun, faire manuellement

<details>
<summary>💡 Voir la réponse</summary>
✅ **Réponse : c** – Avec **15 scripts** et des **dépendances**, il est temps de passer à **Airflow** !
</details>

---

## 📊 Votre score

- **10/10** : 🏆 Expert orchestration !
- **8-9/10** : 🌟 Très bien !
- **6-7/10** : 💪 Bon début !
- **< 6/10** : 📚 Relisez le notebook

---

**Félicitations ! 🎉 Vous savez maintenant orchestrer vos pipelines !**

---

## 📚 Ressources

### Outils
- [Crontab Guru](https://crontab.guru/) — Tester et générer des expressions cron
- [Apache Airflow](https://airflow.apache.org/) — Documentation officielle
- [Airflow Tutorial](https://airflow.apache.org/docs/apache-airflow/stable/tutorial/index.html)

### Alternatives à Airflow

| Outil | Description | Cas d'usage |
|-------|-------------|-------------|
| **Prefect** | Orchestration moderne, Pythonic | Alternative plus simple à Airflow |
| **Dagster** | Data orchestration avec types | Pipelines ML |
| **Luigi** | Par Spotify, simple | Pipelines batch |
| **Mage** | Low-code, moderne | Prototypage rapide |

### Cloud managed

| Cloud | Service |
|-------|--------|
| **AWS** | MWAA (Managed Airflow), Step Functions |
| **GCP** | Cloud Composer (Managed Airflow) |
| **Azure** | Data Factory, Synapse Pipelines |

---

## ➡️ Félicitations !

Tu as terminé le parcours **Data Engineering - From Zero to Hero Niveau** ! 🎉

### Pour continuer ton apprentissage

👉 Passe au module `intermédiare`

---

🎉 **Bravo !** Tu as maintenant toutes les bases pour devenir Data Engineer !